## OP Learning Agenda: SF Class of 2014

A project to determine what, if anything, influenced the graduation success of the San Francisco class of 2014.

In [8]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import statsmodels.api as sm
import numpy as np
from tabulate import tabulate

In [9]:
%matplotlib inline

In [10]:
today = datetime.today()
in_file = Path.cwd() / "data" / "processed" / "processed_data.pkl"
report_dir = Path.cwd() / "reports"
report_file = report_dir / "Excel_Analysis_{today:%b-%d-%Y}.xlsx"

in_file2 = Path.cwd() / "data" / "processed" / "processed_data_file2.pkl"

in_file3 = Path.cwd() / "data" / "processed" / "processed_data_file3.pkl"


In [11]:
df = pd.read_pickle(in_file)

df2 = pd.read_pickle(in_file2)

df3 = pd.read_pickle(in_file3)

In [12]:
def sf_cross_tab(df, column, normalize="index"):
    return pd.crosstab(
        df[df.site == "San Francisco"].high_school_class,
        df[df.site == "San Francisco"][column],
        normalize=normalize,
        margins=True,
    )

##  General Distributions

SF Class of 2014 is on track to have the highest 6 year grad rate, with almost 70% of students already graduating, but that number isn't significantly higher than the class of 2013. Though we do see a reasonably big jump from 2012 to 2013. 



#### Table 1. San Francisco 6 Year Graduation Rate by High School Class 

In [43]:
# Grad Rate Less than 6 years

grad_rate_6_year = sf_cross_tab(df, "graduated_4_year_degree_less_6_years").round(2)

print(tabulate(grad_rate_6_year, headers=["HS Class","% Did not Graduate", "Graduation Rate"], tablefmt='simple'))


HS Class      % Did not Graduate    Graduation Rate
----------  --------------------  -----------------
2011                        0.53               0.47
2012                        0.45               0.55
2013                        0.32               0.68
2014                        0.3                0.7
All                         0.39               0.61


#### Table 2. San Francisco 6 Year Graduation Count by High School Class 


In [42]:
grad_numbers_6_year = sf_cross_tab(df, "graduated_4_year_degree_less_6_years", normalize=False).round(2)

print(tabulate(grad_numbers_6_year, headers=["HS Class","# Did not Graduate", "Graduates", "All"], tablefmt='simple'))


HS Class      # Did not Graduate    Graduates    All
----------  --------------------  -----------  -----
2011                          18           16     34
2012                          19           23     42
2013                          12           25     37
2014                          16           37     53
All                           65          101    166


#### Table 3. San Francisco 5 Year Graduation Rate by High School Class

To be more accurate, we can look at the 5 year grad rate, but this tells essentially the same story.

In [41]:
grad_rate_5_year = sf_cross_tab(df, "graduated_4_year_degree_less_5_years").round(2)

print(tabulate(grad_rate_5_year, headers=["HS Class","% Did not Graduate", "Graduation Rate"], tablefmt='simple'))



HS Class      % Did not Graduate    Graduation Rate
----------  --------------------  -----------------
2011                        0.53               0.47
2012                        0.48               0.52
2013                        0.32               0.68
2014                        0.3                0.7
All                         0.4                0.6


### Statistical Test

If we run an independent t-test on the 5 year graduation rates, we see that the class of 2014 is not statistically higher than the class of 2013, or 2012. 

#### P Value from t-test comparing 2014 -> 2013

In [28]:
population1_test_1 = (
    df[(df.site == "San Francisco") & (df.high_school_class == 2013)][
        "graduated_4_year_degree_less_5_years"
    ]
).values


population2_test_1 = (
    df[(df.site == "San Francisco") & (df.high_school_class == 2014)][
        "graduated_4_year_degree_less_5_years"
    ]
).values

In [29]:
# p value of independent t-test on populations above
round(sm.stats.ttest_ind(population1_test_1, population2_test_1)[1],2)


0.82

#### P Value from t-test comparing 2014 -> 2012 

In [31]:
population1_test_2 = (
    df[(df.site == "San Francisco") & (df.high_school_class == 2012)][
        "graduated_4_year_degree_less_5_years"
    ]
).values


population2_test_2 = (
    df[(df.site == "San Francisco") & (df.high_school_class == 2014)][
        "graduated_4_year_degree_less_5_years"
    ]
).values

In [32]:
round(sm.stats.ttest_ind(population1_test_2, population2_test_2)[1],2)


0.08

#### P Value from t-test comparing 2014 -> 2011
Note, this value is < 0.5

In [33]:
population1_test_3 = (
    df[(df.site == "San Francisco") & (df.high_school_class == 2011)][
        "graduated_4_year_degree_less_5_years"
    ]
).values


population2_test_3 = (
    df[(df.site == "San Francisco") & (df.high_school_class == 2014)][
        "graduated_4_year_degree_less_5_years"
    ]
).values

In [35]:
round(sm.stats.ttest_ind(population1_test_3, population2_test_3)[1],2)


0.03

### Other Distributions

Based on the above results, I don't believe we can say that the class of 2014 was notably higher that previous classes. It does however appear there is an upswring in graduation rate that has been increasing since 2011, with a decent jump from 2012 -> 2013 (though not a statistically significant one).

With that in mind, here are other notable differences in the high school class distributions which might indicatate changes that are influencing the graduation rate 

#### 11th Grade College Eligibility GPA

This is the most notable distrobution change, with the class of 2014 having by far the highest 11th grade GPAs, with almost 75% of that student group having over a 3.0. Compared to 53% from the class of 2013 (the next highest)

In [40]:
gpa_buckets = sf_cross_tab(df, "gpa_bucket").round(2)

gpa_buckets

print(tabulate(gpa_buckets, headers=["HS Class","2.5 - 2.74", "2.5 or less","2.75 - 2.9", "3.0 - 3.49", "3.5 or greater"], tablefmt='simple'))


HS Class      2.5 - 2.74    2.5 or less    2.75 - 2.9    3.0 - 3.49    3.5 or greater
----------  ------------  -------------  ------------  ------------  ----------------
2011                0.06           0.5           0.12          0.12              0.21
2012                0.02           0.43          0.1           0.21              0.24
2013                0.19           0.14          0.14          0.24              0.3
2014                0.09           0.13          0.04          0.25              0.49
All                 0.09           0.28          0.09          0.21              0.33


In [44]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>